In [1]:
import pyro
from pyro.distributions import LogNormal, Categorical, MultivariateNormal, Normal
import torch
import pyro.poutine as poutine
import pandas as pd
import numpy as np
import logging

## Understanding shapes

### Torch Tensors
Torch Tensors have shapes. These are important for example when it comes to broadcasting.

#### Broadcasting
Broadcasting heavily relies on shapes

In [2]:
#### Simple examples for broadcasting
A = torch.ones(2, 3)
A.shape #10,3
B = torch.ones(2, 3)
A + B

tensor([[2., 2., 2.],
        [2., 2., 2.]])

How can I add one to the first row in A and 2 to the second

In [3]:
B = torch.tensor([[1],[2]]) #2,1 Tensor
A + B

tensor([[2., 2., 2.],
        [3., 3., 3.]])

In [10]:
dd = torch.ones(3,4,5) + torch.ones(5)
dd.shape

torch.Size([3, 4, 5])

For more complex examples, see https://pytorch.org/docs/master/notes/broadcasting.html

###  Distributions have additional shapes

In order to tell, with (parts of) Variables are independ, distributions have addtional shapes.

In [13]:
B.shape #[2,1]
dist = Normal(B*1,1) #These are independend
dist.batch_shape, dist.event_shape

(torch.Size([2, 1]), torch.Size([]))

In [5]:
dist1 = MultivariateNormal(torch.zeros((2)),torch.eye(2,2))
dist1.batch_shape, dist1.event_shape

(torch.Size([]), torch.Size([2]))

The batch shape describes independ Variables. The variables in the Event shape are (or better can be) depended.

Samples from distribution

In [8]:
b = pyro.sample("b", dist1)
b #b.batch_shape

tensor([0.9579, 0.0612])

## Debugging models


In [76]:
def model(X, y):
    N = X.shape[0]
    b = pyro.sample("b", Normal(torch.zeros(3), torch.ones(3)).to_event(1))
    s = pyro.sample("s", LogNormal(torch.ones(3), 0.2*torch.ones(3)).to_event(1))
    sigma = pyro.sample("sigma", LogNormal(0, 1))
    with pyro.plate("data", N):
        mu = (((X - b)*s) ** 2).sum(dim=1).sqrt()
    return pyro.sample("obs", Normal(mu, sigma), obs=y)

X = torch.randn((4,3))
y = torch.randn(4,1)

pyro.clear_param_store()
traced_model = poutine.trace(model)
trace = poutine.trace(traced_model).get_trace(X, y)
trace.compute_log_prob()
print(trace.format_shapes())

print("-------------     ")
pyro.clear_param_store()
auto_guide = pyro.infer.autoguide.AutoDelta(model)
adam = pyro.optim.Adam({"lr": 0.02})  # Consider decreasing learning rate.
#pyro.render_model(auto_guide, model_args=(X, y))
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)
loss = svi.step(X, y)

Trace Shapes:        
 Param Sites:        
Sample Sites:        
       b dist     | 3
        value     | 3
     log_prob     |  
       s dist     | 3
        value     | 3
     log_prob     |  
   sigma dist     |  
        value     |  
     log_prob     |  
    data dist     |  
        value   4 |  
     log_prob     |  
     obs dist   4 |  
        value 4 1 |  
     log_prob 4 4 |  
-------------     


ValueError: at site "obs", invalid log_prob shape
  Expected [], actual [4, 4]
  Try one of the following fixes:
  - enclose the batched tensor in a with pyro.plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions

VI relies on the shapes to be correct, while sampling the code above works, it's not possible to do variational inference, in the example above. To check the shapes, the following is possible

In [66]:
import pyro.poutine as poutine
traced_model = poutine.trace(model)
trace = poutine.trace(traced_model).get_trace(X, y)
trace.compute_log_prob()
print(trace.format_shapes())

Trace Shapes:        
 Param Sites:        
Sample Sites:        
       b dist     | 3
        value     | 3
     log_prob     |  
       s dist     | 3
        value     | 3
     log_prob     |  
   sigma dist     |  
        value     |  
     log_prob     |  
    data dist     |  
        value   4 |  
     log_prob     |  
     obs dist 4 4 |  
        value   4 |  
     log_prob 4 4 |  


In [72]:
pyro.clear_param_store()
auto_guide = pyro.infer.autoguide.AutoDelta(model)
adam = pyro.optim.Adam({"lr": 0.02})  # Consider decreasing learning rate.
#pyro.render_model(auto_guide, model_args=(X, y))
elbo = pyro.infer.Trace_ELBO()
svi = pyro.infer.SVI(model, auto_guide, adam, elbo)
loss = svi.step(X, y)

ValueError: at site "obs", invalid log_prob shape
  Expected [], actual [4]
  Try one of the following fixes:
  - enclose the batched tensor in a with pyro.plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions